# Setup Jupyter Notebook
To be able to change code and see changes in the notebook, we need to set the notebook to reload modules.
Also, since we use the code in src, we need to add the src folder to the path.

In [ ]:
import os
import sys

# Set jupyter to reload modules automatically so we can modify the code and see the changes without restarting the kernel
%load_ext autoreload
%autoreload 2

# Add the parent directory to the path so we can import the modules
parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)
sys.path.append('src')
sys.path.append(parent_directory + '/src')

# Data Settings

In [ ]:
# name of the binvox file
name = "name_of_the_shape_to_bound"
type = "tet_or_voxel" # tet or voxel

# definition volume. Set to None to use the axis aligned bounding box of the object
lower_point = [-1., -1., -1.]
upper_point = [ 1.,  1.,  1.]

# use pretraining or not
use_pretraining = False

# scale factor for scaling the object to a different size whitin the definition volume
scale_factor = 1.0

# dataloader settings
number_of_train_test_points = 4_000_000
if type == "voxel":
    num_initial_splits_for_voxel = 4

# batch size during data generation to avoid memory overflow
batch_size_spec = 256 

In [ ]:
if type == "voxel":
    buffer = False
elif type == "tet":
    buffer = True

# Ensure all Training- and Testdata is existing

In [ ]:
from src.point_dataloader import PointDataloader
from src.tet_mesh import TetrahedralMesh

# For tets the points are buffered
if type == "tet":
    mesh = TetrahedralMesh(vtk_file="../tet_meshes/" + name + ".vtk")

    p1 = PointDataloader(mesh=mesh, target_number_of_points=number_of_train_test_points, batch_size_spec=batch_size_spec, outside=True, type="train", lower_point=lower_point, upper_point=upper_point, buffer=buffer, scale_factor=scale_factor)
    p2 = PointDataloader(mesh=mesh, target_number_of_points=number_of_train_test_points, batch_size_spec=batch_size_spec, outside=True, type="test",  lower_point=lower_point, upper_point=upper_point, buffer=buffer, scale_factor=scale_factor)
    del p1, p2

    if use_pretraining:
        p3 = PointDataloader(mesh=mesh, target_number_of_points=number_of_train_test_points, batch_size_spec=batch_size_spec, outside=False, type="train", lower_point=lower_point, upper_point=upper_point, buffer=buffer, scale_factor=scale_factor)
        p4 = PointDataloader(mesh=mesh, target_number_of_points=number_of_train_test_points, batch_size_spec=batch_size_spec, outside=False, type="test",  lower_point=lower_point, upper_point=upper_point, buffer=buffer, scale_factor=scale_factor)
        del p3, p4

    del mesh

# Start Timing

In [ ]:
import time
start_time_all = time.time()

# Load Mesh

In [ ]:
from src.tet_mesh import TetrahedralMesh
from src.cube_mesh import CubeMesh

if type == "tet":
    mesh = TetrahedralMesh(vtk_file="../tet_meshes/" + name + ".vtk", scale_factor=scale_factor)
    print("Number of Tetrahedra", len(mesh.tetrahedra))
elif type == "voxel":
    mesh = CubeMesh(binvox_file="../binvox/" + name + ".binvox")
    print("Number of boxes", len(mesh.upper_points))

In [ ]:
# Show the defined shape

# import matplotlib.pyplot as plt
# from src.visualization import create_plt_axes

# ax = create_plt_axes(upper_point=upper_point, lower_point=lower_point)
# if type == "tet":
#     mesh.render_shape(ax)
# elif type == "voxel":
#     mesh.render(ax)
#     pass

# mesh.render_bounding_box(ax)
# # set view direction to neg y  
# ax.view_init(elev=90, azim=0)
# #plt.axis('equal')
# plt.show()

# Create Randomly Initialized Model

In [ ]:
# model settings
layer_width=50
num_hidden_layers=2
random_initialized_model_path = name + "_random_initialized_model.npz"

from src.model import Model

# Depending on whether the AABB of the Mesh or a predifined volume is used, the lower and upper point of the model is set to the AABB of the mesh or the predifined volume
lower_point_model = lower_point
upper_point_model = upper_point
if lower_point_model is None and upper_point_model is None:
    lower_point_model = mesh.lower_point
    upper_point_model = mesh.upper_point

model = Model(lower_point_model, upper_point_model, layer_width=layer_width, num_hidden_layers=num_hidden_layers) # Do not inflate (by giving no affine dataloader)

print("Lower point definition volume model:", model.lower_point)
print("Upper point definition volume model:", model.upper_point)

model.save_as_npz_file(random_initialized_model_path)

print("Model size:", model.get_number_of_parameters())

In [ ]:
# Show initial state of random initialized model


# import matplotlib.pyplot as plt
# from src.visualization import create_plt_axes

# ax = create_plt_axes()
# model.render_with_points(ax, 1_000_000)
# ax.set_title("random initialized model")
# plt.show()

# Load Point Datasets

In [ ]:
point_dataloader_train_outside = PointDataloader(mesh=mesh, target_number_of_points=number_of_train_test_points, batch_size_spec=batch_size_spec, outside=True, type="train", lower_point=lower_point, upper_point=upper_point, buffer=buffer)
point_dataloader_test_outside  = PointDataloader(mesh=mesh, target_number_of_points=number_of_train_test_points, batch_size_spec=batch_size_spec, outside=True, type="test",  lower_point=lower_point, upper_point=upper_point, buffer=buffer)

if use_pretraining:
    point_dataloader_train_inside  = PointDataloader(mesh=mesh, target_number_of_points=number_of_train_test_points, batch_size_spec=batch_size_spec, outside=False, type="train", lower_point=lower_point, upper_point=upper_point, buffer=buffer)
    point_dataloader_test_inside   = PointDataloader(mesh=mesh, target_number_of_points=number_of_train_test_points, batch_size_spec=batch_size_spec, outside=False, type="test",  lower_point=lower_point, upper_point=upper_point, buffer=buffer)

In [ ]:
# Show Cut from Data Loader

# import matplotlib.pyplot as plt
# from src.visualization import create_plt_axes, Limits

# if use_pretraining:
#     limits_y = Limits(x_limits=[-1, 1], y_limits=[-0.05, 0.05], z_limits=[-1, 1])
#     limits_x = Limits(x_limits=[-0.05, 0.05], y_limits=[-1, 1], z_limits=[-1, 1])

#     dataloaders =[point_dataloader_train_inside, point_dataloader_train_outside, point_dataloader_test_inside, point_dataloader_test_outside]
#     headlines = ["point_dataloader_train_inside", "point_dataloader_train_outside", "point_dataloader_test_inside", "point_dataloader_test_outside"]
#     for loader, headline in zip (dataloaders, headlines):
#         ax = create_plt_axes()
#         loader.render(ax, limits=limits_y)
#         ax.view_init(elev=0, azim=-90)
#         ax.set_title(headline + ", y-axis")
#         plt.show()

#         ax = create_plt_axes()
#         loader.render(ax, limits=limits_x)
#         ax.view_init(elev=0, azim=0)
#         ax.set_title(headline + ", x-axis")
#         plt.show()

# Pretraining

In [ ]:
# Settings
batch_size_pretraining = 32_768

start_lr = 0.1
lr_factor = 0.5
num_epoch_factor = 1.2
start_num_epochs = 5
lr_reduction_depth = 10

In [ ]:
if use_pretraining:
    # addapt dataloader settings for pretraining
    dataloaders = [point_dataloader_train_outside, point_dataloader_train_inside]
    for i, dataloader in enumerate(dataloaders):
        dataloader.batch_size = batch_size_pretraining
        print("batch_size dataloader ", i, ":", dataloader.batch_size)

    lrs = []
    num_epochs = []
    current_lr = start_lr
    for i in range(lr_reduction_depth):
        lrs.append(current_lr * (lr_factor ** i))
        num_epochs.append(int(start_num_epochs * (num_epoch_factor ** i)))
    print("Learn Rates:", lrs)
    print("Epoch Numbers:", num_epochs)
    print("Total Epochs:", sum(num_epochs))

    from src.trainer import PointTrainer
    best_model_path_pretraining = name + "_pretrained.npz"

    trainer = PointTrainer(point_dataloader_outside_train=point_dataloader_train_outside,
                           point_dataloader_outside_test=point_dataloader_test_outside,
                           point_dataloader_inside_train=point_dataloader_train_inside,
                           point_dataloader_inside_test=point_dataloader_test_inside,
                           mesh=mesh)

    model = trainer.train(model=model,
                          best_model_path=best_model_path_pretraining,
                          lrs=lrs,
                          num_epochs=num_epochs)
    
    model = Model(lower_point_model, upper_point_model, path=best_model_path_pretraining)

    trainer.plot_training()

# Create Affine Dataloader

In [ ]:
from src.affine_dataloader import AffineDataloader

batch_size_affine_dataloader = 2048

if type == "tet":
    affine_dataloader = AffineDataloader(mesh=mesh, batch_size=batch_size_affine_dataloader)
elif type == "voxel":
    affine_dataloader = AffineDataloader(mesh=mesh, batch_size=batch_size_affine_dataloader, num_initial_splits_for_voxel=num_initial_splits_for_voxel)
print("Number of samples in affine dataloader: ", affine_dataloader.get_num_samples())

In [ ]:
# Show general boxes

# import matplotlib.pyplot as plt
# from src.visualization import create_plt_axes

# ax = create_plt_axes()
# affine_dataloader.render(ax)
# plt.show()

# Inflation

In [ ]:
from src.model import Model

if use_pretraining:
    model = Model(lower_point_model, upper_point_model, path=best_model_path_pretraining, affine_dataloader=affine_dataloader)
else:
    model = Model(lower_point_model, upper_point_model, path=random_initialized_model_path, affine_dataloader=affine_dataloader)

inflated_model_path = name + "_inflated.npz"
model.save_as_npz_file(inflated_model_path)

# Optimization

In [ ]:
# settings

# auto determine lambda and lr settings
lambda_start  = 1
lambda_factor = 1.3
start_lr = 0.001
lr_reduction_factor = 0.7
max_tries = 50

# training settings
lr_factor = 0.7
num_epoch_factor = 1.0
epochs_per_lr = 500
lr_reduction_depth = 4

In [ ]:
# adapt the batch size to the number of samples in the affine dataloader
num_batches_affine_dataloader = len(affine_dataloader)
point_dataloader_train_outside.change_num_batches(num_batches_affine_dataloader)

print("Number of batches affine dataloader:", len(affine_dataloader))
print("New number of batches point dataloader:", len(point_dataloader_train_outside))

print("Batch size affine dataloader:", affine_dataloader.batch_size)
print("New batch size for point dataloader:", point_dataloader_train_outside.batch_size)

## Train

In [ ]:
import traceback
from src.model import Model
from src.trainer import OptimizationTrainer
from src.evaluation import test_with_dataloader_points

final_best_model_path_optimization = name + "_optimized.npz"
intermediate_best_model_path = name + "_best_model.npz"

# Initialize intermediate best and final best model with the inflated model
model = Model(lower_point_model, upper_point_model, path=inflated_model_path)

model.save_as_npz_file(intermediate_best_model_path)
model.save_as_npz_file(final_best_model_path_optimization)

print("Number of parameters in model: ", model.get_number_of_parameters())

# Initial best accuracy with inflated model
best_found_acc, _ = test_with_dataloader_points(model, point_dataloader_test_outside)

trainer = OptimizationTrainer(box_dataloader=affine_dataloader, train_dataloader_outside=point_dataloader_train_outside, test_dataloader_outside=point_dataloader_test_outside, mesh=mesh)

cnt_failed_runs = 0
current_lr = start_lr
success = False
# while there was no success, try again.
while not success and cnt_failed_runs < max_tries:
    # create lr and num_epochs lists for the current run
    lrs = []
    num_epochs = []
    for i in range(lr_reduction_depth):
        lrs.append(current_lr * (lr_factor ** i))
        num_epochs.append(int(epochs_per_lr * (num_epoch_factor ** i)))

    print("lrs: ", lrs)
    print("num_epochs: ", num_epochs)

    # Load inflated model
    model = Model(lower_point_model, upper_point_model, path=inflated_model_path)

    # numerical errors can cause the learning to crash (e.g. by producing NaN weights). Except and try again
    try:
        # training
        trainer.train(model=model, best_model_path=intermediate_best_model_path, lrs=lrs, num_epochs=num_epochs, lambda_start=lambda_start, lambda_increase_factor=lambda_factor, max_lambda_increases=max_tries)
        success = True
    except Exception as e:
        print(traceback.format_exc())
        cnt_failed_runs += 1
    
    model = Model(lower_point_model, upper_point_model, path=intermediate_best_model_path)
    accuracy, _ = test_with_dataloader_points(model, point_dataloader_test_outside)

    print("Accuracy from last training:", accuracy, "Best found Accuracy: ", accuracy)
    
    if accuracy > best_found_acc:
        best_found_acc = accuracy
        model.save_as_npz_file(final_best_model_path_optimization)
        print("Saved model as: ", final_best_model_path_optimization)
    
    current_lr = current_lr * lr_reduction_factor

os.remove(intermediate_best_model_path)

model = Model(lower_point_model, upper_point_model, path=final_best_model_path_optimization)

trainer.plot_training()

# Lambda Reduction

In [ ]:
# lambda reduction settings
lambda_reduction_depth = 10
lambda_reduction_epochs = 50
lambda_reduction_factor = 0.8

In [ ]:
import time
import os
from src.trainer import LambdaReductionTrainer

start_time_lambda_reduction = time.time()

intermediate_best_model_path = name + "_best_model.npz"
final_best_model_path_lambda_reduction = name + "_lambda_reduced.npz"

# Load the last lambda and lr values from the optimization trainer
current_lambda=trainer.current_lambda
print("last lambda: ", current_lambda)
last_lr = lrs[-1]
print("last lr: ", last_lr)

# create lr and num_epochs lists for the current run
lrs = []
num_epochs = []
lrs.append(last_lr)
num_epochs.append(int(lambda_reduction_epochs/2))
lrs.append(last_lr * 0.5)
num_epochs.append(int(lambda_reduction_epochs/2))

# create the trainer
trainer = LambdaReductionTrainer(box_dataloader=affine_dataloader,
                                train_dataloader_outside=point_dataloader_train_outside,
                                test_dataloader_outside=point_dataloader_test_outside,
                                mesh=mesh)

# Load the best found model from the optimization
model = Model(lower_point_model, upper_point_model, path=final_best_model_path_optimization)
model.save_as_npz_file(final_best_model_path_lambda_reduction)

found_new_best = False

# inizialize the best found accuracy with the accuracy of the model before the lambda reduction
best_found_acc, _ = test_with_dataloader_points(model, point_dataloader_test_outside)

for i in range(lambda_reduction_depth):
    current_lambda *= lambda_reduction_factor
    print("Round", i+1, "of", lambda_reduction_depth, "new lambda:", current_lambda)

    # In case the model gets numerically instable, catch the exception and continue with the next lambda value
    try:
        model = trainer.train(model=model,
                              best_model_path=intermediate_best_model_path,
                              lrs=lrs,
                              num_epochs=num_epochs, 
                              lambda_start=current_lambda)
    except Exception as e:
        print(traceback.format_exc())
        print("Training stopped due to error. Continue with next lambda value.")
    
    model = Model(lower_point_model, upper_point_model, path=intermediate_best_model_path)

    accuracy, _ = test_with_dataloader_points(model, point_dataloader_test_outside)
    print("Best found Accuracy: ", accuracy)
    
    if accuracy > best_found_acc:
        best_found_acc = accuracy
        model.save_as_npz_file(final_best_model_path_lambda_reduction)
        found_new_best = True
        print("Saved model as: ", final_best_model_path_lambda_reduction)

model = Model(lower_point_model, upper_point_model, path=final_best_model_path_lambda_reduction)

os.remove(intermediate_best_model_path)
    
print("Time for lambda reduction: ", (time.time() - start_time_lambda_reduction) / 60, "min, found new best: ", found_new_best)

# Save Final Model

In [ ]:
# Create unique model name for identification
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")
save_model_path = name + "_" + dt_string + ".npz"
print(save_model_path)

# Load best model
model = Model(lower_point_model, upper_point_model, path=final_best_model_path_lambda_reduction)

# save model in npz format (giving additional information about the architecture)
model.save_as_npz_file(save_model_path)

# Save Time for full training

In [ ]:
total_time = time.time() - start_time_all
print("Total time: ", total_time, "s")
print("Total time: ", total_time / 60, "min")

with open("times.txt", "a") as file:
    file.write(name + " " + str(total_time) + " s " + save_model_path + "\n")

# Create Validation Points

In [ ]:
number_of_val_points = 5_000_000

from src.validation_points import ValidationPoints
validation_points = ValidationPoints(mesh=mesh, number_of_points=number_of_val_points, batch_size_spec=batch_size_spec, buffer=buffer, lower_point=lower_point, upper_point=upper_point, scale_factor=scale_factor)

In [ ]:
# Show Cut from Validation Points

# import matplotlib.pyplot as plt
# from src.visualization import create_plt_axes, Limits

# limits_y = Limits(x_limits=[-1, 1], y_limits=[-0.05, 0.05], z_limits=[-1, 1])
# limits_x = Limits(x_limits=[-0.05, 0.05], y_limits=[-1, 1], z_limits=[-1, 1])

# ax = create_plt_axes()
# validation_points.render(ax, limits=limits_y)
# ax.view_init(elev=0, azim=-90)
# ax.set_title("validation_points, y-axis")
# plt.show()

# ax = create_plt_axes()
# validation_points.render(ax, limits=limits_x)
# ax.view_init(elev=0, azim=0)
# ax.set_title("validation_points, x-axis")
# plt.show()

# Evaluate Models

In [ ]:
from src.evaluation import determine_FPR_FNR, sanity_check_with_validation_points, test_with_parallelepipeds

def test_model(path, check_for_false_negatives):
    model_to_test = Model(lower_point_model, upper_point_model, path=path)
    determine_FPR_FNR(model_to_test, validation_points)

    if check_for_false_negatives:
        false_negatives, _ =test_with_parallelepipeds(model_to_test, affine_dataloader)
        assert false_negatives == 0, "There are false negatives in the inflated model"
        print("No false negatives found.")

        passed_sanity_check = sanity_check_with_validation_points(model_to_test, validation_points)
        assert passed_sanity_check, "Sanity check failed"
        print("Sanity check passed")

In [ ]:
# Evaluate random initialized model. This model can have false negatives.
#test_model(random_initialized_model_path, check_for_false_negatives=False)

In [ ]:
# Evaluate pretrained model. This model can have false negatives.
if use_pretraining:
    test_model(best_model_path_pretraining, check_for_false_negatives=False)

In [ ]:
# Evaluate inflated model. This model must not have false negatives.
test_model(inflated_model_path, check_for_false_negatives=True)

In [ ]:
# Evaluate optimized model. This model must not have false negatives.
#test_model(final_best_model_path_optimization, check_for_false_negatives=True)

In [ ]:
# Evaluate model after lambda reduction.
test_model(final_best_model_path_lambda_reduction, check_for_false_negatives=True)

In [ ]:
from src.evaluation import test_accuracy_with_validation_points

model = Model(lower_point_model, upper_point_model, path=final_best_model_path_lambda_reduction)

accuracy, points_inside_bounding_volume_div_points_inside_object = test_accuracy_with_validation_points(model, validation_points)
print("Accuracy: ", accuracy)
print("Points inside bounding volume div points inside object: ", points_inside_bounding_volume_div_points_inside_object)

#write to result.txt if file exists
with open("results.txt", "a") as file:
    file.write(name + " " + str(best_found_acc) + " " + str(points_inside_bounding_volume_div_points_inside_object) + " " + save_model_path + "\n")

# Visualize Resulting Models

In [ ]:
# Settings
view_axis = "y"

In [ ]:
import matplotlib.pyplot as plt
from src.visualization import create_plt_axes, create_plt_axes_2d

def show_model(path, title):
    model_to_visualize = Model(lower_point_model, upper_point_model, path=path)
    
    ax = create_plt_axes()
    model_to_visualize.render_with_points(ax, num_points=2_000_000, set_view_axis=view_axis)
    ax.set_title(title)
    plt.show()

    plt_axes = create_plt_axes_2d()
    model_to_visualize.show_difference_to_mesh_2d(plt_axes, 1_000_000, axis="y", axis_value=0.0, mesh_to_show_difference_to=mesh, buffer=buffer, scale_factor=scale_factor)
    plt.show()

In [ ]:
#Show random initialized model
show_model(random_initialized_model_path, "random initialized model")

In [ ]:
# Show pretrained model
if use_pretraining:
    show_model(best_model_path_pretraining, "pretrained model")

In [ ]:
# Show inflated model
show_model(inflated_model_path, "inflated model")

In [ ]:
# Show optimized model
show_model(final_best_model_path_optimization, "optimized model")

In [ ]:
# Show model after lambda reduction
show_model(final_best_model_path_lambda_reduction, "model after lambda reduction")